# Задание 3.1 - Примеры задач анализа текстов

In [58]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer, TfidfVectorizer
from sklearn.cross_validation import cross_val_score
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB

Считайте датасет в Python (можете сразу грузить все в память, выборка небольшая), выясните, что используется в качестве разделителей и как проставляются метки классов.

In [7]:
data = pd.read_csv("SMSSpamCollection.txt", sep = '\t', names = ('spam_or_ham', 'text'))
data.head()

,spam_or_ham,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


Подготовьте для дальнейшей работы два списка: список текстов в порядке их следования в датасете и список соответствующих им меток классов. В качестве метки класса используйте 1 для спама и 0 для "не спама".

In [12]:
texts = data.text
labels = [1 if data.spam_or_ham[i] == 'spam' else 0 for i in range(data.shape[0])]
print texts[0]
print labels[:3]

Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...
[0, 0, 1]


Используя sklearn.feature_extraction.text.CountVectorizer со стандартными настройками, получите из списка текстов матрицу признаков X.

In [14]:
?CountVectorizer

In [23]:
c_v = CountVectorizer()
X = c_v.fit_transform(texts)
X

<5572x8713 sparse matrix of type '<type 'numpy.int64'>'
	with 74169 stored elements in Compressed Sparse Row format>

Оцените качество классификации текстов с помощью LogisticRegression() с параметрами по умолчанию, используя sklearn.cross_validation.cross_val_score и посчитав среднее арифметическое качества на отдельных fold'ах. Установите random_state=2. Параметр cv задайте равным 10. В качестве метрики качества используйте f1-меру. Получившееся качество - один из ответов, которые потребуются при сдаче задания. Ответ округлить до 1 знака после запятой.

In [17]:
?cross_val_score

In [28]:
random_state=2
model = LogisticRegression()
ans1 =  str("%.1f" % cross_val_score(model, X, labels, scoring = 'f1', cv = 10).mean())
print ans1

0.9


In [29]:
with open('ans1.txt', 'w') as f:
    f.write(ans1)

 А теперь обучите классификатор на всей выборке и спрогнозируйте с его помощью класс для следующих сообщений:

Прогнозы классификатора (0 - не спам, 1 - спам), записанные через пробел, будут ответом в одном из вопросов ниже.

In [37]:
test = ["FreeMsg: Txt: CALL to No: 86888 & claim your reward of 3 hours talk time to use from your phone now! Subscribe6GB",
        "FreeMsg: Txt: claim your reward of 3 hours talk time",
        "Have you visited the last lecture on physics?",
        "Have you visited the last lecture on physics? Just buy this book and you will have all materials! Only 99$",
        "Only 99$"]

In [47]:
lg_pipeline = Pipeline(
            [("vectorizer", CountVectorizer()),
            ("classifier", LogisticRegression())])

In [48]:
lg_pipeline.fit(texts, labels)

Pipeline(memory=None,
     steps=[('vectorizer', CountVectorizer(analyzer=u'word', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
    ...ty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))])

In [54]:
test_pr = lg_pipeline.predict(test)
print test_pr

[1 1 0 0 0]


In [55]:
with open('ans2.txt', 'w') as f:
    ans2 = ' '.join(map(str, test_pr))
    f.write(ans2)

Задайте в CountVectorizer параметр ngram_range=(2,2), затем ngram_range=(3,3), затем ngram_range=(1,3). Во всех трех случаях измерьте получившееся в кросс-валидации значение f1-меры, округлите до второго знака после точки, и выпишете результаты через пробел в том же порядке. В данном эксперименте мы пробовали добавлять в признаки n-граммы для разных диапазонов n - только биграммы, только триграммы, и, наконец, все вместе - униграммы, биграммы и триграммы. Обратите внимание, что статистики по биграммам и триграммам намного меньше, поэтому классификатор только на них работает хуже. В то же время это не ухудшает результат сколько-нибудь существенно, если добавлять их вместе с униграммами, т.к. за счет регуляризации линейный классификатор не склонен сильно переобучаться на этих признаках.

In [56]:
n_ranges = [(2, 2), (3, 3), (1, 3)]
ans = []
for n_range in n_ranges:
    lg_pipeline = Pipeline(
            [("vectorizer", CountVectorizer(ngram_range = n_range)),
            ("classifier", LogisticRegression())])
    ans.append(cross_val_score(lg_pipeline, texts, labels, scoring = 'f1', cv = 10).mean())

ans

[0.8224220664187133, 0.7250161555467377, 0.9251382558648837]

In [57]:
with open('ans3.txt', 'w') as f:
    ans3 = ' '.join(map(str, ['%.2f' % sc for sc in ans]))
    f.write(ans3)

Повторите аналогичный п.7 эксперимент, используя вместо логистической регрессии MultinomialNB(). Обратите внимание, насколько сильнее (по сравнению с линейным классификатором) наивный Байес страдает от нехватки статистики по биграммам и триграммам.

In [59]:
# Так почему-то не работает (через pipeline)
n_ranges = [(2, 2), (3, 3), (1, 3)]
ans = []
for n_range in n_ranges:
    lg_pipeline = Pipeline(
            [("vectorizer", CountVectorizer(ngram_range = n_range)),
            ("classifier", MultinomialNB())])
    ans.append(cross_val_score(lg_pipeline, texts, labels, scoring = 'f1', cv = 10).mean())

ans

[0.9330215115853413, 0.871265305963816, 0.9472991994136064]

In [62]:
n_ranges = [(2, 2), (3, 3), (1, 3)]
ans = []
for n_range in n_ranges:
    c_v = CountVectorizer(ngram_range = n_range)
    X = c_v.fit_transform(texts)
    model = MultinomialNB()
    ans.append(cross_val_score(model, X, labels, scoring = 'f1', cv = 10).mean())

ans

[0.6455015177985443, 0.37871948524573595, 0.8884859656061002]

In [63]:
with open('ans4.txt', 'w') as f:
    ans4 = ' '.join(map(str, ['%.2f' % sc for sc in ans]))
    f.write(ans4)

Попробуйте использовать в логистической регрессии в качестве признаков Tf*idf из TfidfVectorizer на униграммах. Повысилось или понизилось качество на кросс-валидации по сравнению с CountVectorizer на униграммах? (напишите в файле с ответом 1, если повысилось, -1, если понизилось, и 0, если изменилось не более чем на 0.01). Обратите внимание, что результат перехода к tf*idf не всегда будет таким - если вы наблюдаете какое-то явление на одном датасете, не надо сразу же его обобщать на любые данные.

In [67]:
lg_pipeline_Tfidf = Pipeline(
            [("vectorizer", TfidfVectorizer()),
            ("classifier", LogisticRegression())])
lg_pipeline_CV = Pipeline(
            [("vectorizer", CountVectorizer()),
            ("classifier", LogisticRegression())])
score_Tfidf = cross_val_score(lg_pipeline_Tfidf, texts, labels, scoring = 'f1', cv = 10).mean()
score_CV = cross_val_score(lg_pipeline_CV, texts, labels, scoring = 'f1', cv = 10).mean()
print ("TfidfTransformer: %.4f" % score_Tfidf)
print ("CountVectorizer: %.4f" % score_CV)
print ("Разница: %.2f" % (score_Tfidf - score_CV))

TfidfTransformer: 0.8785
CountVectorizer: 0.9326
Разница: -0.05


In [68]:
with open('ans5.txt', 'w') as f:
    ans5 = str(-1)
    f.write(ans5)